Word Similarity: To create word represenation that can take semantics into consideration.<br>Unconstrained : Consider that the constraints are removed and allowed to use any data or model.


In [ ]:
pip install allennlp


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.2/730.2 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.2/248.2 kB 31.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 62.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 37.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/

In [ ]:
pip install allennlp==2.1.0 allennlp-models==2.1.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.1/585.1 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.9/407.9 kB 28.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of allennlp to determine which version is compatible with other requirements. This could take a while.
ERROR: Ignored the following versions that require a different python version: 0.2.0 Requires-Python ==3.6
ERROR: Could not find a version that satisfies the requirement torch<1.8.0,>=1.6.0 (from allennlp) (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2)
ERROR: No matching distribution found for torch<1.8.0,>=1.6.0


In [ ]:
from allennlp.modules.elmo import Elmo, batch_to_ids
import torch


options_file = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_options.json"
weight_file = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_weights.hdf5"


elmo = Elmo(options_file, weight_file, num_output_representations=1, dropout=0)

def get_elmo_embedding(word):

    sentences = [[word.lower()]]  # ELMo is case-sensitive
    character_ids = batch_to_ids(sentences)

    embeddings = elmo(character_ids)

    word_embedding = embeddings['elmo_representations'][0][0, 0, :].detach().numpy()
    return word_embedding


embedding = get_elmo_embedding("apple")
print(embedding)


Output()

Output()

[ 0.7557074  -0.29991007  0.74354494 ...  0.1482637   0.46911323
  0.6117441 ]


In [ ]:
import pandas as pd
simlex_data = pd.read_csv('SimLex-999.txt', sep='\t')

In [ ]:
embedding1 = get_elmo_embedding("old")
embedding2 = get_elmo_embedding("new")

In [ ]:
from numpy.linalg import norm
import numpy as np

def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (norm(vec1) * norm(vec2))

In [ ]:
similarity = cosine_similarity(embedding1, embedding2)

In [ ]:
def get_word_vectors_sim(word1, word2):
    
    word1_vector = get_elmo_embedding(word1)
    word2_vector = get_elmo_embedding(word2)

 
    similarity = cosine_similarity(word1_vector, word2_vector)
    scaled = 5 * (similarity + 1) 
    return scaled


In [ ]:

simlex_data['scaled_cosine_similarity'] = simlex_data.apply(lambda row: get_word_vectors_sim(row['word1'], row['word2']), axis=1)

In [ ]:
from sklearn.metrics import mean_squared_error
from scipy.stats import spearmanr


In [ ]:
mse = mean_squared_error(simlex_data['SimLex999'], simlex_data['scaled_cosine_similarity'])
print(f"Mean Squared Error (MSE): {mse}")


Mean Squared Error (MSE): 15.7962279055215


In [ ]:
rmse = mean_squared_error(simlex_data['SimLex999'], simlex_data['scaled_cosine_similarity'], squared=False)
print(f"Root Mean Squared Error (MSE): {rmse}")

Root Mean Squared Error (MSE): 3.9744468678699807


In [ ]:
spearman_corr, _ = spearmanr(simlex_data['SimLex999'], simlex_data['scaled_cosine_similarity'])
print(f"Spearman's Rank Correlation: {spearman_corr}")


Spearman's Rank Correlation: 0.4173345288139787


In [ ]:
#ERROR ANALYSIS

In [ ]:
simlex_data['squared_error'] = (simlex_data['SimLex999'] - simlex_data['scaled_cosine_similarity']) ** 2

In [ ]:

simlex_data_sorted = simlex_data.sort_values(by='squared_error', ascending=False)

In [ ]:

simlex_data_sorted.head(10)


,word1,word2,POS,SimLex999,conc(w1),conc(w2),concQ,Assoc(USF),SimAssoc333,SD(SimLex),scaled_cosine_similarity,squared_error
47,unnecessary,necessary,A,0.63,1.56,1.80,1,0.41,0,0.61,8.608935,63.663397
829,shrink,grow,V,0.23,3.55,3.03,2,0.50,1,1.20,8.206543,63.625234
12,easy,difficult,A,0.58,2.07,2.21,1,2.83,1,0.36,8.555651,63.611014
211,absence,presence,N,0.40,2.31,2.72,1,1.38,1,1.50,8.231058,61.325467
15,smart,dumb,A,0.55,1.75,2.36,1,2.10,1,0.47,8.343512,60.738826
56,easy,tough,A,0.52,2.07,2.50,1,0.27,0,1.87,8.251350,59.773776
80,tiny,huge,A,0.60,3.11,3.54,2,0.15,0,0.59,8.324501,59.667914
50,bad,great,A,0.35,1.68,1.81,1,0.35,0,1.49,8.048247,59.263008
76,dull,funny,A,0.55,2.37,2.50,1,0.20,0,1.76,8.172040,58.095498
36,sad,funny,A,0.95,3.07,2.50,2,0.74,1,1.28,8.565231,57.991747
